In [1]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=1

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=1


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import numpy as np
from ganime.data.base import load_dataset
from ganime.model.p2p.p2p_test import P2P
import tensorflow as tf
from tqdm import tqdm
tf.get_logger().setLevel('ERROR')

In [4]:
train_ds, test_ds, input_shape = load_dataset("moving_mnist_vae", "../../data", batch_size=100)# * strategy.num_replicas_in_sync)

2022-05-03 07:52:36.725381: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-05-03 07:52:37.111645: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 14264 MB memory:  -> device: 0, name: NVIDIA RTX A4000, pci bus id: 0000:25:00.0, compute capability: 8.6


In [5]:
train_ds_iterator = train_ds.as_numpy_iterator()
x = next(train_ds_iterator)

2022-05-03 07:52:38.239689: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


In [21]:
type(test_ds)

tensorflow.python.data.ops.dataset_ops.ShuffleDataset

In [19]:
#with strategy.scope():
model = P2P(z_dim=64)

In [24]:
#with strategy.scope():
model.compile(frame_predictor_optimizer=tf.keras.optimizers.Adam(learning_rate=0.002, beta_1=0.9, beta_2=0.999, epsilon=1e-8),
             posterior_optimizer=tf.keras.optimizers.Adam(learning_rate=0.002, beta_1=0.9, beta_2=0.999, epsilon=1e-8),
             prior_optimizer=tf.keras.optimizers.Adam(learning_rate=0.002, beta_1=0.9, beta_2=0.999, epsilon=1e-8),
             encoder_optimizer=tf.keras.optimizers.Adam(learning_rate=0.002, beta_1=0.9, beta_2=0.999, epsilon=1e-8),
             decoder_optimizer=tf.keras.optimizers.Adam(learning_rate=0.002, beta_1=0.9, beta_2=0.999, epsilon=1e-8),)

history = model.fit(train_ds, epochs=1)

  0%|          | 0/19 [00:00<?, ?it/s]

  0%|          | 0/19 [00:00<?, ?it/s]

90/90 [==============================] - 53s 311ms/step - loss: 8650.0600 - reconstruction_loss: 8653.6475 - kl_loss: 0.7098 - align_loss: 0.1181 - cpc_loss: 0.0425


In [8]:
from ganime.visualization.videos import display_videos
generated = model(x[0])

In [9]:
display_videos(generated, n_rows=1, n_cols=2)

In [10]:
x[0][:,0,...].shape

(100, 64, 64, 1)

In [11]:
h, skip = model.encoder(x[0][:,0,...])
global_z, _ = model.encoder(x[0][:,-1,...])

In [12]:
time_until_cp = tf.fill([100, 1], (19 - 1 + 1) / 19)
delta_time = tf.fill([100, 1], ((1 - 0) / 19))

In [13]:
h_cpaw = tf.concat([h, global_z, time_until_cp, delta_time], axis=-1)

In [14]:
zt_p, _, _ = model.prior(h_cpaw)

In [15]:
h = model.frame_predictor(
    tf.concat([h, zt_p, time_until_cp, delta_time], axis=-1)
)

In [16]:
model.decoder([tf.ones_like(h), [tf.ones_like(s) for s in skip]]).numpy().mean()

0.0

In [17]:
model.decoder([h, skip]).numpy().mean()

0.0